# Merging cell barcodes
Based on the Otsu filtering in notebook #3, we now merge all the barcode sets with significant overlaps. Run the following code in the bash container found in `PUMATAC_dependencies`
Open the bash container:

In [ ]:
img_path=PUMATAC_dependencies/cache/vibsinglecellnf-singlecelltoolkit-2022-07-07-0638c1d.img
singularity exec --cleanenv -H $PWD -B /staging,/lustre1,/data,/vsc-hard-mounts,/scratch $img_path bash

In the container, run the following commands:  
Generate fragments:

In [ ]:
export POLARS_MAX_THREADS=8
export OMP_NUM_THREADS=8

for file in ATACflow_tutorial_preprocessing_out/data/reports/barcard/*tomerge.tsv
do
    echo $file
    sample=(`basename $file`)
    sample=${sample%.tomerge.tsv}
    # echo $sample
    jaccardfile=$file
    infile=ATACflow_tutorial_preprocessing_out/data/fragments/$sample.fragments.raw.tsv.gz
    outfile=ATACflow_tutorial_preprocessing_out/data/fragments/$sample.fragments.tsv
    outfile2=$outfile.gz


    merge_cbs_over_jaccard_index_threshold.py -j $jaccardfile -i $infile -o $outfile &
done

Then bgzip the output:

In [2]:
for file in ATACflow_tutorial_preprocessing_out/data/fragments/*.fragments.tsv
do
    echo $file
    bgzip -f -@ 8 $file &
done

full_preprocessing_out/data/fragments/OHS_s3atac_1.FULL.fragments.tsv
exists
full_preprocessing_out/data/fragments/OHS_s3atac_2.FULL.fragments.tsv
exists


And index the fragments files.

In [ ]:
for file in ATACflow_tutorial_preprocessing_out/data/fragments/*.fragments.tsv.gz
do
    tabix -p bed $file &
done

If you want to process the `.bam` files further, add a new barcode tag `DB` to the corresponding .bam files, based on the barcode translate files

In [ ]:
prog=../0_resources/single_cell_toolkit_rust/target/release/add_new_bc_tag
prog=/dodrio/scratch/projects/starting_2022_023/benchmark/scatac_benchmark/0_resources/single_cell_toolkit_rust/add_new_bc_tag_if_old_found
for file in full_preprocessing_out/data/bam/UCS*possorted.bam
do
    echo $file
    sample=(`basename $file`)
    sample=${sample%_____R1.bwa.out.possorted.bam}
    echo $sample
    outfile=full_preprocessing_out/data/bam/$sample.possorted.dbfixed.bam
    bc_map=full_preprocessing_out/data/fragments/$sample.fragments.tsv.original_CB_to_merged_CBs.tsv
    ls $bc_map
    if [ -f "$outfile" ]; then
        echo "$outfile exists, skipping"
    else 
        echo "$outfile does not exist, generating!"
        $prog $file $outfile $bc_map CB DB &
    fi
done

In [ ]:
for file in full_preprocessing_out/data/bam/*.possorted.dbfixed.bam
do
    echo $file
    outfile=$file.bai
    if [ -f "$outfile" ]; then
        echo "$outfile exists, skipping"
    else 
        echo "$outfile does not exist, generating!"
        samtools index -@ 8 $file &
    fi
done